In [12]:
# 01_explore_fandom_html.ipynb

from pathlib import Path
import random
import textwrap

import pandas as pd
from bs4 import BeautifulSoup

# So pandas prints nicely
pd.set_option("display.max_colwidth", 200)

In [13]:
# Root path of the repo (adjust if you run from somewhere else)
REPO_ROOT = Path("..").resolve()

HTML_DIR = REPO_ROOT / "data" / "raw" / "fandom_html"/"money-heist"
HTML_DIR, HTML_DIR.exists(), len(list(HTML_DIR.glob("*.html")))

(PosixPath('/data/sundeep/Fandom_SI/data/raw/fandom_html/money-heist'),
 True,
 355)

In [14]:
html_files = sorted(HTML_DIR.glob("*.html"))
print(f"Found {len(html_files)} HTML files.")

for p in html_files[:10]:
    print("-", p.name)

Found 355 HTML files.
- money-heist_fandom_com_wiki_48_Meters_Underground.html
- money-heist_fandom_com_wiki_48_Meters_Underground__Part_3_.html
- money-heist_fandom_com_wiki_5_Minutes_Earlier.html
- money-heist_fandom_com_wiki_5_Minutes_Earlier__Part_4_.html
- money-heist_fandom_com_wiki_5_Minutes_Later.html
- money-heist_fandom_com_wiki_5_Minutes_Later__Part_4_.html
- money-heist_fandom_com_wiki_A_Family_Tradition.html
- money-heist_fandom_com_wiki_A_Quick_Vacation.html
- money-heist_fandom_com_wiki_A_Quick_Vacation__Part_3_.html
- money-heist_fandom_com_wiki_After_Love.html


In [15]:
def load_html(path: Path) -> str:
    """Read raw HTML from disk."""
    return path.read_text(encoding="utf-8", errors="ignore")


def parse_html(html: str) -> BeautifulSoup:
    """Parse HTML into BeautifulSoup object."""
    return BeautifulSoup(html, "html.parser")


def summarize_soup(soup: BeautifulSoup) -> dict:
    """Return a small summary dict for a Fandom page."""
    # Title
    title = (soup.title.string or "").strip() if soup.title else ""

    # Headings
    h1_tags = [h.get_text(strip=True) for h in soup.find_all("h1")]
    h2_tags = [h.get_text(strip=True) for h in soup.find_all("h2")]

    # Paragraphs (rough content length)
    paragraphs = [p.get_text(" ", strip=True) for p in soup.find_all("p")]
    num_paragraphs = len(paragraphs)
    total_chars = sum(len(p) for p in paragraphs)

    # Links
    a_tags = soup.find_all("a")
    num_links = len(a_tags)

    # Very basic “is this likely a Fandom page?”
    hint_fandom = "fandom" in title.lower() or any(
        "fandom" in (a.get("href") or "").lower() for a in a_tags[:50]
    )

    return {
        "title": title,
        "num_h1": len(h1_tags),
        "h1_sample": h1_tags[:3],
        "num_h2": len(h2_tags),
        "h2_sample": h2_tags[:5],
        "num_paragraphs": num_paragraphs,
        "total_chars_in_p": total_chars,
        "num_links": num_links,
        "looks_like_fandom": hint_fandom,
    }

In [16]:
N_SAMPLES = 5

sample_files = random.sample(html_files, k=min(N_SAMPLES, len(html_files)))
sample_files

[PosixPath('/data/sundeep/Fandom_SI/data/raw/fandom_html/money-heist/money-heist_fandom_com_wiki_Berlin_s_Monastery.html'),
 PosixPath('/data/sundeep/Fandom_SI/data/raw/fandom_html/money-heist/money-heist_fandom_com_wiki_David_Barrocal.html'),
 PosixPath('/data/sundeep/Fandom_SI/data/raw/fandom_html/money-heist/money-heist_fandom_com_wiki_Shakir.html'),
 PosixPath('/data/sundeep/Fandom_SI/data/raw/fandom_html/money-heist/money-heist_fandom_com_wiki_Alfonso.html'),
 PosixPath('/data/sundeep/Fandom_SI/data/raw/fandom_html/money-heist/money-heist_fandom_com_wiki_Stockholm.html')]

In [17]:
rows = []
for path in sample_files:
    html = load_html(path)
    soup = parse_html(html)
    summary = summarize_soup(soup)
    summary["file_name"] = path.name
    rows.append(summary)

df_samples = pd.DataFrame(rows)[[
    "file_name",
    "title",
    "looks_like_fandom",
    "num_h1",
    "h1_sample",
    "num_h2",
    "h2_sample",
    "num_paragraphs",
    "total_chars_in_p",
    "num_links",
]]
df_samples

,file_name,title,looks_like_fandom,num_h1,h1_sample,num_h2,h2_sample,num_paragraphs,total_chars_in_p,num_links
0,money-heist_fandom_com_wiki_Berlin_s_Monastery.html,Monastery of San Giovanni | Money Heist Wiki | Fandom,True,1,[Monastery of San Giovanni],4,"[Monastery of San Giovanni, Trivia[], References[], ]",6,745,450
1,money-heist_fandom_com_wiki_David_Barrocal.html,David Barrocal | Money Heist Wiki | Fandom,True,1,[David Barrocal],2,"[David Barrocal, ]",5,343,433
2,money-heist_fandom_com_wiki_Shakir.html,Shakir | Money Heist Wiki | Fandom,True,1,[Shakir],15,"[Shakir, Biographical Information, Relationships, Professional Information, Series Information]",7,716,461
3,money-heist_fandom_com_wiki_Alfonso.html,Alfonso | Money Heist Wiki | Fandom,True,1,[Alfonso],13,"[Alfonso, Biographical Information, Relationships, Professional Information, Contents]",6,805,470
4,money-heist_fandom_com_wiki_Stockholm.html,Mónica Gaztambide | Money Heist Wiki | Fandom,True,1,[Mónica Gaztambide],15,"[Stockholm, Biographical Information, Relationships, Professional Information, Series Information]",41,12641,664


In [18]:
# Choose one file to inspect in depth
inspect_path = sample_files[0]  # or html_files[0] or a specific Path
inspect_path

PosixPath('/data/sundeep/Fandom_SI/data/raw/fandom_html/money-heist/money-heist_fandom_com_wiki_Berlin_s_Monastery.html')

In [19]:
raw_html = load_html(inspect_path)
soup = parse_html(raw_html)

print("=== File ===")
print(inspect_path.name)
print("\n=== Title ===")
print(soup.title.string.strip() if soup.title else "NO TITLE")

print("\n=== First 400 chars of RAW HTML ===")
print(textwrap.shorten(raw_html.replace("\n", " "), width=400, placeholder=" ..."))

# Grab first few paragraphs of visible text
paragraphs = [p.get_text(" ", strip=True) for p in soup.find_all("p")]
clean_text = "\n\n".join(paragraphs[:5])

print("\n=== First 5 paragraphs (cleaned) ===")
print(textwrap.shorten(clean_text, width=800, placeholder=" ..."))

=== File ===
money-heist_fandom_com_wiki_Berlin_s_Monastery.html

=== Title ===
Monastery of San Giovanni | Money Heist Wiki | Fandom

=== First 400 chars of RAW HTML ===
<!DOCTYPE html> <html class="client-nojs sse-other l2u-other" lang="en" dir="ltr"> <head> <meta charset="UTF-8"> <title>Monastery of San Giovanni | Money Heist Wiki | Fandom</title> <script>document.documentElement.className="client-js sse-other l2u- ...

=== First 5 paragraphs (cleaned) ===
Money Heist Wiki Hey there! 👋 Are you a fan of Money Heist ? Click here to join our Discord server and connect with other fans! 🥳 ( see more ) READ MORE Monastery of San Giovanni Inhabitant(s) Berlin , The Professor Location Italy Filming location The Hermitage of San Frutos (Saint Fructus) Abbey de Parraces, Segovia (Interior and Courtyard) First appearance We're Back The Monastery of San Giovanni is the Italian monastery where Berlin lived in the years leading up the heist of the Royal Mint of Spain . He planned the heist of the